The SK model:
### Sherrington–Kirkpatrick (SK) Spin Glass Model

The Hamiltonian of the Sherrington–Kirkpatrick model is

$$
\mathcal{H}_N(\boldsymbol{\sigma})
= - \sum_{1 \le i < j \le N} J_{ij}\,\sigma_i \sigma_j
\;-\;
\sum_{i=1}^{N} h_i\,\sigma_i ,
$$

where the terms are defined as follows:

- $\sigma_i \in \{+1,-1\}$ are **Ising spins**, representing binary degrees of freedom.
- $J_{ij}$ are **quenched random couplings** between spins $i$ and $j$, drawn independently from a Gaussian distribution
  $$
  J_{ij} \sim \mathcal{N}\!\left(0,\frac{J^2}{N}\right),
  $$
  often with $J^2 = 1$.
- The scaling $1/N$ ensures a **well-defined thermodynamic limit**, so that the total energy is extensive (of order $N$).
- $h_i$ is an **external field** acting on spin $i$ (often taken as $h_i = h$ or $h_i = 0$).
- The first term encodes **frustrated interactions** due to random signs of $J_{ij}$.
- The second term biases spins toward alignment with the external field.

The SK model is a **mean-field spin glass**, meaning every spin interacts with every other spin with random strength.
